In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [2]:
x,y = make_moons(n_samples=500, noise=0.30, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)

In [3]:
lr = LogisticRegression(solver='lbfgs',random_state=42)
rfc = RandomForestClassifier(n_estimators=150, random_state=42)
svm_clf = SVC(gamma='scale', random_state=42, probability=True)

In [4]:
def build_nn():
    model = Sequential([Dense(50, activation='relu', input_shape=[2]),
                       Dense(1, activation='sigmoid')])
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
keras_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs=500, verbose=False)

C:\Users\User\AppData\Local\Temp\ipykernel_2876\464239683.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(build_nn, epochs=500, verbose=False)


In [7]:
keras_clf._estimator_type = 'classifier'

In [8]:
voting = VotingClassifier(estimators=[('lr',lr), ('rf',rfc), ('svc',svm_clf), ('keras', keras_clf)], voting='soft', flatten_transform=True)

In [9]:
voting.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf',
                              RandomForestClassifier(n_estimators=150,
                                                     random_state=42)),
                             ('svc', SVC(probability=True, random_state=42)),
                             ('keras',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002242D84B3A0>)],
                 voting='soft')

In [11]:
for clf in (lr, rfc, svm_clf, keras_clf, voting):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
4/4 [==============================] - 0s 2ms/step
KerasClassifier 0.888
4/4 [==============================] - 0s 2ms/step
VotingClassifier 0.912
